# Transform and insert data

In [17]:
!pip install mysql-connector-python
!pip install pymysql

/bin/bash: /opt/anaconda/envs/Python3/lib/libtinfo.so.6: no version information available (required by /bin/bash)

         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.

bash: /opt/anaconda/envs/Python3/lib/libtinfo.so.6: no version information available (required by bash)


## Transform Simply Recipes

In [76]:
def transform_simplyrecipes_json(recipe_json):
    transformed_recipe = {
        "title": recipe_json["title"],
        "instructions": " ".join(recipe_json["instructions"]),
        "ingredients": [{"name": ingredient} for ingredient in recipe_json["ingredients"]]
    }
    return transformed_recipe

In [77]:
import json

# Load the simplyrecipes.json file
with open("../output/simplyrecipes.json") as f:
    simplyrecipes_data = json.load(f)

# Transform the data
transformed_data_simplyrecipes = [transform_simplyrecipes_json(recipe) for recipe in simplyrecipes_data]

# The transformed_data list now contains dictionaries ready for insertion into the MySQL database

## Transform Edamam

In [78]:
def transform_edamam_data(raw_data):
    transformed_data = []

    for recipe in raw_data:
        nutrients = recipe["totalNutrients"]

        nutrition = {
            "calories": round(recipe["calories"]),
            "protein": round(nutrients["PROCNT"]["quantity"]) if "PROCNT" in nutrients else None,
            "fat": round(nutrients["FAT"]["quantity"]) if "FAT" in nutrients else None,
            "carbohydrates": round(nutrients["CHOCDF"]["quantity"]) if "CHOCDF" in nutrients else None,
            "sugar": round(nutrients["SUGAR"]["quantity"]) if "SUGAR" in nutrients else None,
            "fiber": round(nutrients["FIBTG"]["quantity"]) if "FIBTG" in nutrients else None,
            "cholesterol": round(nutrients["CHOLE"]["quantity"]) if "CHOLE" in nutrients else None,
            "sodium": round(nutrients["NA"]["quantity"]) if "NA" in nutrients else None
        }

        transformed_item = {
            "title": recipe["label"],
            "source_url": recipe["url"],
            "image_url": recipe["image"],
            "instructions": "",
            "ingredients": [{"name": ing} for ing in recipe["ingredientLines"]],
            "nutrition": nutrition,
        }

        transformed_data.append(transformed_item)

    return transformed_data

In [79]:
import json

# Load the data
with open("../output/edamam-output.json", "r") as f:
    edamam_data = json.load(f)

# Transform the data
transformed_data_edamam = transform_edamam_data(edamam_data)

## Transform Tasty

In [93]:
def transform_tastyapi_data(raw_data):
    transformed_data = []

    for item in raw_data:
        transformed_recipe = {
            "title": item["name"],
            "image_url": item["thumbnail_url"],
        }

        if "instructions" in item:
            transformed_recipe["instructions"] = "\n".join([instruction["display_text"] for instruction in item["instructions"]])

        if "sections" in item:
            transformed_recipe["ingredients"] = [{"name": ing["raw_text"]} for section in item["sections"] for ing in section["components"]]

        if "nutrition" in item:
            transformed_recipe["nutrition"] = {
                "calories": item["nutrition"].get("calories", None),
                "protein": item["nutrition"].get("protein", None),
                "fat": item["nutrition"].get("fat", None),
                "carbohydrates": item["nutrition"].get("carbohydrates", None),
                "sugar": item["nutrition"].get("sugar", None),
                "fiber": item["nutrition"].get("fiber", None),
            }

        transformed_data.append(transformed_recipe)

    return transformed_data


In [94]:
# Load the data
with open("../output/tasty-output.json", "r") as f:
    tastyapi = json.load(f)

transformed_data_tastyapi = transform_tastyapi_data(tastyapi)

In [95]:
print(transformed_data_tastyapi)

[{'title': 'Chocolate Chip Cookie Dough Blended Oats', 'image_url': 'https://img.buzzfeed.com/thumbnailer-prod-us-east-1/video-api/assets/432886.jpg', 'instructions': 'Add the oats, milk, egg, baking powder, cinnamon, maple syrup, 1 tablespoon of almond butter, the banana, and vanilla, if using, to a blender and blend until smooth. If using nuts and cacao nibs, add to the blender and blend for just a few seconds to help distribute without breaking them up too much. This will add a lovely crunch to your oats.\nPour half the batter into a large mug or oven-safe ramekin, greased with butter or cooking spray. Add the remaining tablespoon of almond butter and 1½ tablespoons of chocolate chips to the center. Cover with the rest of the batter. Top with the remaining 1½ tablespoons of chocolate chips.\nPreheat the air fryer to 325°F (160°C). Place the mug in the air fryer and cook for 20 minutes, until golden brown and a toothpick inserted into the center comes out clean. Alternatively, microw

## Transform TheMealDB

In [103]:
def transform_themealdb_data(raw_data):
    transformed_data = []

    for item in raw_data:
        transformed_recipe = {
            "title": item["strMeal"],
            "image_url": item["strMealThumb"],
            "instructions": item["strInstructions"].replace("\r\n", "\n"),
            "ingredients": [],
        }

        for i in range(1, 21):
            ingredient_key = f"strIngredient{i}"
            measure_key = f"strMeasure{i}"

            if item[ingredient_key] and item[ingredient_key].strip():
                transformed_recipe["ingredients"].append({
                    "name": item[ingredient_key].strip(),
                    "quantity": item[measure_key].strip() if item[measure_key] and item[measure_key].strip() else None
                })

        transformed_data.append(transformed_recipe)

    return transformed_data

In [104]:
with open("../output/themealdb-output.json", "r") as f:
    themealdb = json.load(f)

transformed_data_themealdb = transform_themealdb_data(themealdb)

# Insert data

In [80]:
import mysql.connector


def connect_to_mysql():
    connection = mysql.connector.connect(
        host="104.196.180.64",
        user="root",
        password="Password-1",
        database="main"
    )
    return connection

In [81]:
import pymysql

def create_mysql_connection():
    connection = pymysql.connect(
        host="104.196.180.64",
        user="root",
        password="Password-1",
        database="main"
    )
    return connection

## Simply Recipes

In [82]:
def insert_simplyrecipes_data(recipe_data):
    for recipe in recipe_data:
        # Insert recipe and get recipe_id
        recipe_id = insert_recipe(recipe)

        # Insert instructions
        insert_instructions(recipe["instructions"], recipe_id)

        # Insert ingredients
        ingredient_ids = insert_ingredients(recipe["ingredients"])

        # Insert recipe_ingredient
        insert_recipe_ingredients(recipe_id, ingredient_ids, recipe["ingredients"])


In [83]:
def insert_recipe(recipe_data):
    connection = create_mysql_connection()
    recipe_id = None

    try:
        with connection.cursor() as cursor:
            sql_query = """
            INSERT INTO recipe (title, instructions)
            VALUES (%s, %s);
            """

            cursor.execute(sql_query, (recipe_data.get("title", None), recipe_data.get("instructions", None)))
            recipe_id = cursor.lastrowid
        connection.commit()
    finally:
        connection.close()

    return recipe_id


def insert_ingredients(ingredients):
    connection = create_mysql_connection()
    ingredient_ids = []

    try:
        with connection.cursor() as cursor:
            for ingredient in ingredients:
                sql_query = """
                INSERT INTO ingredient (name)
                VALUES (%s);
                """

                cursor.execute(sql_query, (ingredient.get("name", None),))
                ingredient_ids.append(cursor.lastrowid)
        connection.commit()
    finally:
        connection.close()

    return ingredient_ids


def insert_instructions(instructions, recipe_id):
    connection = create_mysql_connection()

    try:
        with connection.cursor() as cursor:
            for position, instruction in enumerate(instructions.split('\n')):
                sql_query = """
                INSERT INTO instruction (recipe_id, position, text)
                VALUES (%s, %s, %s);
                """

                cursor.execute(sql_query, (recipe_id, position, instruction.strip() if instruction else None))
        connection.commit()
    finally:
        connection.close()


def insert_recipe_ingredients(recipe_id, ingredient_ids, ingredients):
    connection = create_mysql_connection()

    try:
        with connection.cursor() as cursor:
            for ingredient_id, ingredient in zip(ingredient_ids, ingredients):
                sql_query = """
                INSERT INTO recipe_ingredient (recipe_id, ingredient_id, quantity, unit)
                VALUES (%s, %s, %s, %s);
                """

                cursor.execute(sql_query, (recipe_id, ingredient_id, ingredient.get("quantity", None), ingredient.get("unit", None)))
        connection.commit()
    finally:
        connection.close()
        
def insert_nutrition(nutrition_data, recipe_id):
    connection = create_mysql_connection()

    try:
        with connection.cursor() as cursor:
            sql_query = """
            INSERT INTO nutrition (
                recipe_id, calories, protein, fat, carbohydrates, sugar, fiber, cholesterol, sodium
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
            """

            cursor.execute(sql_query, (
                recipe_id,
                nutrition_data.get("calories", None),
                nutrition_data.get("protein", None),
                nutrition_data.get("fat", None),
                nutrition_data.get("carbohydrates", None),
                nutrition_data.get("sugar", None),
                nutrition_data.get("fiber", None),
                nutrition_data.get("cholesterol", None),
                nutrition_data.get("sodium", None)
            ))
        connection.commit()
    finally:
        connection.close()

In [84]:
insert_simplyrecipes_data(transformed_data_simplyrecipes)

## Edamam

In [85]:
def insert_edamam_data(recipe_data):
    for recipe in recipe_data:
        # Insert recipe and get recipe_id
        recipe_id = insert_recipe(recipe)

        # Insert instructions
        if "instructions" in recipe:
            insert_instructions(recipe["instructions"], recipe_id)

        # Insert ingredients
        ingredient_ids = insert_ingredients(recipe["ingredients"])

        # Insert recipe_ingredient
        insert_recipe_ingredients(recipe_id, ingredient_ids, recipe["ingredients"])

        # Insert nutrition data
        if "nutrition" in recipe:
            insert_nutrition(recipe["nutrition"], recipe_id)

In [86]:
insert_edamam_data(transformed_data_edamam)

## TastyAPI

In [98]:
def insert_tastyapi_data(recipe_data):
    for recipe in recipe_data:
        # Insert recipe and get recipe_id
        recipe_id = insert_recipe(recipe)

        # Insert instructions
        if "instructions" in recipe:
            insert_instructions(recipe["instructions"], recipe_id)

        # Insert ingredients
        if "ingredients" in recipe:
            ingredient_ids = insert_ingredients(recipe["ingredients"])

            # Insert recipe_ingredient
            insert_recipe_ingredients(recipe_id, ingredient_ids, recipe["ingredients"])

        # Insert nutrition data
        if "nutrition" in recipe:
            insert_nutrition(recipe["nutrition"], recipe_id)

In [99]:
insert_tastyapi_data(transformed_data_tastyapi)

## TheMealDB

In [105]:
def insert_themealdb_data(recipe_data):
    for recipe in recipe_data:
        # Insert recipe and get recipe_id
        recipe_id = insert_recipe(recipe)

        # Insert instructions
        if "instructions" in recipe:
            insert_instructions(recipe["instructions"], recipe_id)

        # Insert ingredients
        ingredient_ids = insert_ingredients(recipe["ingredients"])

        # Insert recipe_ingredient
        insert_recipe_ingredients(recipe_id, ingredient_ids, recipe["ingredients"])

In [106]:
insert_themealdb_data(transformed_data_themealdb)

# Verify

In [107]:
def fetch_data_from_table(table_name):
    connection = create_mysql_connection()
    data = []

    try:
        with connection.cursor() as cursor:
            sql_query = f"SELECT * FROM {table_name};"
            cursor.execute(sql_query)
            data = cursor.fetchall()
    finally:
        connection.close()

    return data


In [108]:
recipe_data = fetch_data_from_table("recipe")
ingredient_data = fetch_data_from_table("ingredient")
instruction_data = fetch_data_from_table("instruction")
nutrition_data = fetch_data_from_table("nutrition")
recipe_ingredient_data = fetch_data_from_table("recipe_ingredient")